In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
%matplotlib inline
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
# importar el arbol de desicion 
from sklearn.tree import DecisionTreeRegressor
# visualizar el arbol de desiciones
import graphviz
# exportar grafo a graphviz
from sklearn.tree import export_graphviz

# Ensemble
---
ensemble = utilizan submodelos y hacen la prediccions

## Decision tree

In [3]:
model = DecisionTreeRegressor(max_depth=2)

x = pd.read_csv('x_opening.csv')
y = x['worldwide_gross']
x = x.drop('worldwide_gross', axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1)
model.fit(x_train, y_train)

treedot = export_graphviz(model, out_file=None, feature_names=x.columns)
treedot

'digraph Tree {\nnode [shape=box] ;\n0 [label="opening_gross <= 41613376.0\\nmse = 4.491994363696598e+16\\nsamples = 1665\\nvalue = 141540319.054"] ;\n1 [label="opening_gross <= 22074047.0\\nmse = 1.33338221931278e+16\\nsamples = 1506\\nvalue = 92999937.199"] ;\n0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;\n2 [label="mse = 4923666241203536.0\\nsamples = 1257\\nvalue = 64781848.271"] ;\n1 -> 2 ;\n3 [label="mse = 3.1478131019987544e+16\\nsamples = 249\\nvalue = 235450289.735"] ;\n1 -> 3 ;\n4 [label="opening_gross <= 70351576.0\\nmse = 1.103981187159362e+17\\nsamples = 159\\nvalue = 601300162.289"] ;\n0 -> 4 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;\n5 [label="mse = 4.067538845924499e+16\\nsamples = 92\\nvalue = 440868287.554"] ;\n4 -> 5 ;\n6 [label="mse = 1.2226485798747635e+17\\nsamples = 67\\nvalue = 821594676.851"] ;\n4 -> 6 ;\n}'

In [4]:
# graphviz.Source(treedot)

## Random forest
usa decision tree  
baging (bootstrap agregation) = de dos datasets sacar dos datasets de igual tamaño, los datos se pueden repetir entre cada dataset y selecciona features diferentes, para sacar diferentes tree y formar un forest

In [5]:
# importar random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

In [6]:
# parametro = cantidad de arboles
forest = RandomForestRegressor(200)
results = cross_validate(forest, x, y, cv=5, return_train_score=True, scoring='r2')

In [7]:
train_scores = results['train_score']
test_scores = results['test_score']
print(np.mean(train_scores))
print(np.mean(test_scores))
# si el train y el test son diferentes, hay overfiting y hay que cambiar features o modelo

0.9660687089704039
0.5203122587293918


## Gradient boosted trees
boosting = ensambla modelos de aprendizaje debil

In [8]:
# importar gradient boosted regresor 
from sklearn.ensemble import GradientBoostingRegressor

In [9]:
ensemble = GradientBoostingRegressor()
results = cross_validate(forest, x, y, cv=5, return_train_score=True, scoring='r2')

train_scores = results['train_score']
test_scores = results['test_score']
print(np.mean(train_scores))
print(np.mean(test_scores))
# si el train y el test son diferentes, hay overfiting y hay que cambiar features o modelo

0.9652012837595254
0.5163705838716179


## Optimizacion de hiperparametros
grid = busqueda exhaustiva  
grid seach = metodo automatico de seleccion de features, buscando la mejor combinacion de features para el modelo  
grid search con cross validation = mide los parametros de forma automatica   
Devalidacion = tomar el dataset train, separarlo en una parte pequeña para medir scores mientras se hace el modelo y asi poder tener el modelo de test libre para medir scores finales

In [17]:
from sklearn.model_selection import train_test_split
# importar grid search con cross validation
from sklearn.model_selection import GridSearchCV

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1)

# parametro = numero de modelos de aprendizaje debil para entrenar el modelo
param_test1 = {'n_estimators': range(20, 501, 20)}
list(param_test1['n_estimators'])

[20,
 40,
 60,
 80,
 100,
 120,
 140,
 160,
 180,
 200,
 220,
 240,
 260,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 420,
 440,
 460,
 480,
 500]

In [15]:
estimador = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=500, min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10)

In [18]:
gsearch1 = GridSearchCV(estimador, param_grid=param_test1, scoring='r2', cv=5)

In [19]:
gsearch1.fit(x_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=8,
                                                 max_features='sqrt',
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=50,
                                                 min_samples_split=500,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter_no_change=None,
            

In [25]:
lista_parametros = list(zip(gsearch1.cv_results_['mean_test_score'], gsearch1.cv_results_['std_test_score'], gsearch1.cv_results_['params']))
lista_parametros

[(0.6553377167287445, 0.05763723230388878, {'n_estimators': 20}),
 (0.7194707183052496, 0.06255974658122894, {'n_estimators': 40}),
 (0.7347239263041044, 0.06359927761595936, {'n_estimators': 60}),
 (0.7389339055742862, 0.062364772765764416, {'n_estimators': 80}),
 (0.7420485206288181, 0.06271425977283825, {'n_estimators': 100}),
 (0.7459322395926498, 0.06235500960463639, {'n_estimators': 120}),
 (0.7495406754127887, 0.06335029572288611, {'n_estimators': 140}),
 (0.7508197586274827, 0.06304688375599073, {'n_estimators': 160}),
 (0.7525654539309128, 0.0634404813604329, {'n_estimators': 180}),
 (0.753490598609423, 0.06446653281824145, {'n_estimators': 200}),
 (0.7545692717605135, 0.06341510679393178, {'n_estimators': 220}),
 (0.7553059694284988, 0.06488882352873111, {'n_estimators': 240}),
 (0.755171486028867, 0.06572082397934954, {'n_estimators': 260}),
 (0.753885223323653, 0.06495415209551623, {'n_estimators': 280}),
 (0.7546023133872947, 0.06568746888772702, {'n_estimators': 300}),
 (

In [27]:
# imprimir mejor parametro
max(lista_parametros)

(0.7553059694284988, 0.06488882352873111, {'n_estimators': 240})

In [33]:
# obtener mejor estimador automaticamente gsearch1.best_estimator_
final_results = cross_validate(gsearch1.best_estimator_,  x_train, y_train, return_train_score=True)

In [34]:
train_scores = final_results['train_score']
test_scores = final_results['test_score']
print(np.mean(train_scores))
print(np.mean(test_scores))
# si el train y el test son diferentes, hay overfiting y hay que cambiar features o modelo

0.8286608694466742
0.7553059694284988


In [35]:
estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=500, min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10, n_estimators=240)

In [36]:
estimator.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=8,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=50, min_samples_split=500,
                          min_weight_fraction_leaf=0.0, n_estimators=240,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=10, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [37]:
estimator.score(x_test, y_test)

0.8092888852563106